In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# this just to make sure we are using only on CPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
import copy
import time

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

from learning_wavelets.datasets import im_dataset_bsd68
from learning_wavelets.utils.metrics import metrics_original_from_ds, metrics_wavelets_from_ds


                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=1.3.0   - required | 1.4.1     installed
numpy          : >=1.16.4  - required | 1.17.4    installed
matplotlib     : >=3.0.0   - required | 3.1.2     installed
astropy        : >=3.0.0   - required | 3.2.3     installed
nibabel        : >=2.3.2   - required | 2.5.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.4.0   - required | 1.4.1     installed
scikit-learn   : >=0.19.1  - requi

In [4]:
np.random.seed(0)

In [5]:
noise_stds = [5, 15, 20, 25, 30, 50, 75]

In [6]:
noise_std_metrics = {}
n_samples = None
for noise_std in tqdm_notebook(noise_stds, 'Noise stds'):
    metrics = []
    im_ds = im_dataset_bsd68(
        mode='testing', 
        batch_size=1, 
        patch_size=None, 
        noise_std=noise_std, 
        return_noise_level=False,
        n_pooling=5,
        n_samples=n_samples,
    )
    metrics.append(('original', metrics_original_from_ds(im_ds)))
        

#     metrics.append(('bm3d', metrics_bm3d(im_gen_test)))
    im_ds = im_dataset_bsd68(
        mode='testing', 
        batch_size=1, 
        patch_size=None, 
        noise_std=noise_std, 
        return_noise_level=False,
        n_pooling=5,
        n_samples=n_samples,
    )
    metrics.append(('wavelets_24', metrics_wavelets_from_ds(im_ds, '24', noise_std=noise_std)))
#     metrics.sort(key=lambda x: x[1].metrics['PSNR'].mean())
    noise_std_metrics[noise_std] = metrics

/volatile/home/Zaccharie/workspace/understanding-unets/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/volatile/home/Zaccharie/workspace/understanding-unets/learning_wavelets/evaluate.py:29: UserWarning: DEPRECATED: skimage.measure.compare_psnr has been moved to skimage.metrics.peak_signal_noise_ratio. It will be removed from skimage.measure in version 0.18.
  return compare_psnr(gt, pred, data_range=1)
/volatile/home/Zaccharie/workspace/understanding-unets/learning_wavelets/evaluate.py:38: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  gt, pred, multichannel=True, data_range=1


In [7]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [8]:
noise_std_metrics

{5: [('original', PSNR = 34.15 +/- 0.02942 SSIM = 0.8855 +/- 0.102),
  ('wavelets_24', PSNR = 35.76 +/- 3.875 SSIM = 0.9457 +/- 0.04561)],
 15: [('original', PSNR = 24.61 +/- 0.03905 SSIM = 0.5689 +/- 0.2587),
  ('wavelets_24', PSNR = 29.56 +/- 5.105 SSIM = 0.8233 +/- 0.1266)],
 20: [('original', PSNR = 22.11 +/- 0.0289 SSIM = 0.4665 +/- 0.266),
  ('wavelets_24', PSNR = 28.25 +/- 5.291 SSIM = 0.7766 +/- 0.1443)],
 25: [('original', PSNR = 20.18 +/- 0.03237 SSIM = 0.3885 +/- 0.2562),
  ('wavelets_24', PSNR = 27.32 +/- 5.368 SSIM = 0.7372 +/- 0.1589)],
 30: [('original', PSNR = 18.59 +/- 0.02857 SSIM = 0.3281 +/- 0.2398),
  ('wavelets_24', PSNR = 26.61 +/- 5.387 SSIM = 0.7033 +/- 0.1717)],
 50: [('original', PSNR = 14.16 +/- 0.03357 SSIM = 0.1834 +/- 0.167),
  ('wavelets_24', PSNR = 24.79 +/- 5.346 SSIM = 0.6031 +/- 0.2038)],
 75: [('original', PSNR = 10.63 +/- 0.03114 SSIM = 0.1013 +/- 0.1035),
  ('wavelets_24', PSNR = 23.46 +/- 5.148 SSIM = 0.5223 +/- 0.2186)]}

In [9]:
# PSNR table
psnr_metrics_table = pd.DataFrame(
    columns=['noise_std'] + ['original', 'wavelets_24', 'bm3d'],
)
for i, (noise_std, metrics) in enumerate(noise_std_metrics.items()):
    psnr_metrics_table.loc[i, 'noise_std'] = noise_std
    for name, m in metrics:
        psnr_metrics_table.loc[i, name] = "{mean:.4} ({std:.4})".format(
            mean=m.metrics['PSNR'].mean(), 
            std=m.metrics['PSNR'].stddev(),
        )
psnr_metrics_table

,noise_std,original,wavelets_24,bm3d
0,5,34.15 (0.01471),35.76 (1.937),NaN
1,15,24.61 (0.01953),29.56 (2.553),NaN
2,20,22.11 (0.01445),28.25 (2.645),NaN
3,25,20.18 (0.01619),27.32 (2.684),NaN
4,30,18.59 (0.01429),26.61 (2.694),NaN
5,50,14.16 (0.01679),24.79 (2.673),NaN
6,75,10.63 (0.01557),23.46 (2.574),NaN


In [10]:
# SSIM table
ssim_metrics_table = pd.DataFrame(
    columns=['noise_std'] + ['original', 'wavelets_24', 'bm3d'],
)
for i, (noise_std, metrics) in enumerate(noise_std_metrics.items()):
    ssim_metrics_table.loc[i, 'noise_std'] = noise_std
    for name, m in metrics:
        ssim_metrics_table.loc[i, name] = "{mean:.4} ({std:.4})".format(
            mean=m.metrics['SSIM'].mean(), 
            std=m.metrics['SSIM'].stddev(),
        )
ssim_metrics_table

,noise_std,original,wavelets_24,bm3d
0,5,0.8855 (0.05101),0.9457 (0.02281),NaN
1,15,0.5689 (0.1293),0.8233 (0.06329),NaN
2,20,0.4665 (0.133),0.7766 (0.07217),NaN
3,25,0.3885 (0.1281),0.7372 (0.07944),NaN
4,30,0.3281 (0.1199),0.7033 (0.08587),NaN
5,50,0.1834 (0.0835),0.6031 (0.1019),NaN
6,75,0.1013 (0.05177),0.5223 (0.1093),NaN
